In [1]:
import pandas as pd
import numpy as np
from tabulate import tabulate
import json
import os

# Getting the Names of the retrieved nodes

In [17]:
def get_filtered_and_sorted_filenames(directory, keywords):
    # Get all files and directories in the specified directory
    all_entries = os.listdir(directory)
    # Filter out directories and files containing "additional_data"
    filenames = [entry for entry in all_entries if os.path.isfile(os.path.join(directory, entry)) and "additional_data" not in entry]

    # shorten filenames to the actual query engine description
    short = [name.split("_")[0] for name in filenames]
    names = {}
    for n in range(len(short)):
        names[short[n]] = filenames[n]
    
    # Create a dictionary to map keywords to filenames
    keyword_to_filename = {keyword: None for keyword in keywords}

    # Map filenames to the corresponding keyword
    for short, filename in names.items():
        for keyword in keywords:
            if keyword in short:
                keyword_to_filename[keyword] = directory + "/" + filename
                break

    # Sort filenames based on the given order of keywords
    sorted_filenames = [keyword_to_filename[keyword] for keyword in keywords if keyword_to_filename[keyword] is not None]

    return sorted_filenames

In [3]:
def extract_control_keys(control_documents: list, type: str, index: int) -> list:
    # extract either the names or the IDs:
    if type == "name":
        control_keys = [document['Name'] for document in control_documents[index]]
    elif type == "id":
        control_keys = [document['ID'] for document in control_documents[index]]
    else:
        raise ValueError("Type must be either 'name' or 'id'")
    return control_keys

In [4]:
def extract_test_keys(df: pd.DataFrame, columns:list, index: int) -> pd.array:
    nodes = df[columns]
    return nodes.iloc[index].astype(str).values

In [5]:
def compare_nodes(df: pd.DataFrame, control_path: str, columns: list, type: str = "name") -> [float, list[bool]]:
    
    
    # read in the control documents
    with open(control_path, 'r') as file:
        control_documents = json.load(file)
        
    # compare if the test_keys contain all elements of the control_keys
    # I.e. if one of the three retrieved nodes contains the name "Umwelt"
    comparison: list[bool] = []
    for i in range(len(control_documents)):
        control_keys = extract_control_keys(control_documents, type, i)
        test_keys = extract_test_keys(df, columns, i)
        comparison.append(all(s in test_keys for s in control_keys))
        
    # calculate the ratio of true to false
    ratio: float = np.mean(comparison)
    
    return ratio, comparison

In [14]:
keywords = ["base", "rerank", "hybrid", "fusion", "hyde"]
files = get_filtered_and_sorted_filenames("./logs/2024-08-13_11-51-31_gpt-4o-mini_text_retrieval_only", keywords)
  
files

['./logs/2024-08-13_11-51-31_gpt-4o-mini_text_retrieval_only/base_gpt-4o-mini_text_retrieval_only_2024-08-13_11-51-31.csv',
 './logs/2024-08-13_11-51-31_gpt-4o-mini_text_retrieval_only/rerank_gpt-4o-mini_text_retrieval_only_2024-08-13_11-51-31.csv',
 './logs/2024-08-13_11-51-31_gpt-4o-mini_text_retrieval_only/hybrid_gpt-4o-mini_text_retrieval_only_2024-08-13_11-51-31.csv',
 './logs/2024-08-13_11-51-31_gpt-4o-mini_text_retrieval_only/fusion_gpt-4o-mini_text_retrieval_only_2024-08-13_11-51-31.csv',
 './logs/2024-08-13_11-51-31_gpt-4o-mini_text_retrieval_only/hyde_gpt-4o-mini_text_retrieval_only_2024-08-13_11-51-31.csv']

In [15]:
folder = "./logs/2024-08-13_11-51-31_gpt-4o-mini_text_retrieval_only"
keywords = ["base", "rerank", "hybrid", "fusion", "hyde"]
files = get_filtered_and_sorted_filenames(folder, keywords)
for filename in files:
    filename = folder + "/" + filename

control_path = "documents.json"

name_columns = [
    "Node 1 Metadata: Name",
    "Node 2 Metadata: Name",
    "Node 3 Metadata: Name"
]

id_columns = [
    "Node 1 ID",
    "Node 2 ID",
    "Node 3 ID"
]

ratios_name = []
comparisons_name = []
ratios_id = []
comparisons_id = []

for file in files:
    df = pd.read_csv(file, sep=';')
    ratio_name, comparison_name = compare_nodes(df=df, control_path=control_path, columns=name_columns, type="name")
    ratio_id, comparison_id = compare_nodes(df=df, control_path=control_path, columns=id_columns, type="id")
    
    ratios_name.append(ratio_name)
    comparisons_name.append(comparison_name)
    ratios_id.append(ratio_id)
    comparisons_id.append(comparison_id)
    
# printing a nice table
data = [
    ["ratio name"]+ratios_name,
    ["ratio id"]+ratios_id
]

print(tabulate(data, headers=["Text", "Jina", "Multilingual", "Cross"], tablefmt="grid"))

+------------+----------+----------+----------+----------------+---------+
|            |          |     Text |     Jina |   Multilingual |   Cross |
+============+==========+==========+==========+================+=========+
| ratio name | 0.416667 | 0.416667 | 0.416667 |       0.416667 |     0.5 |
+------------+----------+----------+----------+----------------+---------+
| ratio id   | 0.416667 | 0.416667 | 0.25     |       0.333333 |     0.5 |
+------------+----------+----------+----------+----------------+---------+


In [16]:
folder = "./logs/2024-08-13_11-51-31_gpt-4o-mini_text_retrieval_only"
keywords = ["base", "rerank", "hybrid", "fusion", "hyde"]
files = get_filtered_and_sorted_filenames(folder, keywords)

control_path = "documents.json"

name_columns = [
    "Node 1 Metadata: Name",
    "Node 2 Metadata: Name",
    "Node 3 Metadata: Name"
]

id_columns = [
    "Node 1 ID",
    "Node 2 ID",
    "Node 3 ID"
]

ratios_name = []
comparisons_name = []
ratios_id = []
comparisons_id = []

for file in files:
    df = pd.read_csv(file, sep=';')
    ratio_name, comparison_name = compare_nodes(df=df, control_path=control_path, columns=name_columns, type="name")
    ratio_id, comparison_id = compare_nodes(df=df, control_path=control_path, columns=id_columns, type="id")

    ratios_name.append(ratio_name)
    comparisons_name.append(comparison_name)
    ratios_id.append(ratio_id)
    comparisons_id.append(comparison_id)

# printing a nice table
data = [
    ["ratio name"]+ratios_name,
    ["ratio id"]+ratios_id
]

print(tabulate(data, headers=["Base", "Rerank", "Hybrid", "Fusion", "HyDE"], tablefmt="grid"))

+------------+----------+----------+----------+----------+--------+
|            |     Base |   Rerank |   Hybrid |   Fusion |   HyDE |
+============+==========+==========+==========+==========+========+
| ratio name | 0.416667 | 0.416667 | 0.416667 | 0.416667 |    0.5 |
+------------+----------+----------+----------+----------+--------+
| ratio id   | 0.416667 | 0.416667 | 0.25     | 0.333333 |    0.5 |
+------------+----------+----------+----------+----------+--------+


In [34]:
#folder = "./logs/2024-08-19_14-09-05_gpt-4o-mini_multilingual_stsb-distilroberta-base_retrieval_only"
folder = "./logs/2024-08-19_15-28-51_gpt-4o-mini_multilingual_msmarco-MiniLM-L12-en-de-v1_retrieval_only"
keywords = ["base", "rerank", "hybrid"]
files = get_filtered_and_sorted_filenames(folder, keywords)

control_path = "documents.json"

rerank_n = 3
name_columns = []
id_columns = []
for i in range(rerank_n):
    name_columns.append(f"Node {i+1} Metadata: Name")
    id_columns.append(f"Node {i+1} ID")


ratios_name = []
comparisons_name = []
ratios_id = []
comparisons_id = []

for file in files:
    df = pd.read_csv(file, sep=';')
    ratio_name, comparison_name = compare_nodes(df=df, control_path=control_path, columns=name_columns, type="name")
    ratio_id, comparison_id = compare_nodes(df=df, control_path=control_path, columns=id_columns, type="id")

    ratios_name.append(ratio_name)
    comparisons_name.append(comparison_name)
    ratios_id.append(ratio_id)
    comparisons_id.append(comparison_id)

# printing a nice table
data = [
    ["ratio name"]+ratios_name,
    ["ratio id"]+ratios_id
]

print(tabulate(data, headers=["Base", "Rerank", "Hybrid"], tablefmt="grid"))

+------------+--------+----------+----------+
|            |   Base |   Rerank |   Hybrid |
+============+========+==========+==========+
| ratio name |   0.25 | 0.333333 | 0.333333 |
+------------+--------+----------+----------+
| ratio id   |   0.25 | 0.333333 | 0.333333 |
+------------+--------+----------+----------+


In [32]:
comparisons_id[2]

[True, True, True, True, False, True, True, False, False, True, False, False]

# Calculating average times

In [6]:
query_times = df["query_time"]
generating_times = df["generating_time"]
total_times = df["total_time"]

# calculate averages
avg_query_time = np.mean(query_times)
avg_generating_time = np.mean(generating_times)
avg_total_time = np.mean(total_times)

# get maximum
max_query_time = np.max(query_times)
max_generating_time = np.max(generating_times)
max_total_time = np.max(total_times)

# get max ids
max_query_id = query_times.idxmax()
max_generating_id = generating_times.idxmax()
max_total_id = total_times.idxmax()

# get minimum
min_query_time = np.min(query_times)
min_generating_time = np.min(generating_times)
min_total_time = np.min(total_times)

# get min ids
min_query_id = query_times.idxmin()
min_generating_id = generating_times.idxmin()
min_total_id = total_times.idxmin()

In [7]:
# Prepare data for table
data = [
    ["Query Time (s)", avg_query_time, max_query_time, min_query_time, max_query_id, min_query_id],
    ["Generating Time (s)", avg_generating_time, max_generating_time, min_generating_time, max_generating_id, min_generating_id],
    ["Total Time (s)", avg_total_time, max_total_time, min_total_time, max_total_id, min_total_id]
]

# Print table
print(tabulate(data, headers=["Type", "Average", "Maximum", "Minimum", "Max ID", "Min ID"], tablefmt="grid"))

+---------------------+-----------+------------+------------+----------+----------+
| Type                |   Average |    Maximum |    Minimum |   Max ID |   Min ID |
+=====================+===========+============+============+==========+==========+
| Query Time (s)      |  0.018802 |  0.0277135 |  0.0139725 |        9 |        3 |
+---------------------+-----------+------------+------------+----------+----------+
| Generating Time (s) | 38.8156   | 96.108     | 12.0876    |       11 |        2 |
+---------------------+-----------+------------+------------+----------+----------+
| Total Time (s)      | 38.8344   | 96.1283    | 12.1091    |       11 |        2 |
+---------------------+-----------+------------+------------+----------+----------+


# Calculating Token Usage

In [25]:
embedding_tokens = df["embedding_tokens"]
prompt_tokens = df["prompt_tokens"]
completion_tokens = df["completion_tokens"]
total_tokens = df["total_tokens"]

# calculate averages
avg_embedding_tokens = np.mean(embedding_tokens)
avg_prompt_tokens = np.mean(prompt_tokens)
avg_completion_tokens = np.mean(completion_tokens)
avg_total_tokens = np.mean(total_tokens)

# get maximum
max_embedding_tokens = np.max(embedding_tokens)
max_prompt_tokens = np.max(prompt_tokens)
max_completion_tokens = np.max(completion_tokens)
max_total_tokens = np.max(total_tokens)

# get max ids
max_embedding_id = embedding_tokens.idxmax()
max_prompt_id = prompt_tokens.idxmax()
max_completion_id = completion_tokens.idxmax()
max_total_id = total_tokens.idxmax()

# get minimum
min_embedding_tokens = np.min(embedding_tokens)
min_prompt_tokens = np.min(prompt_tokens)
min_completion_tokens = np.min(completion_tokens)
min_total_tokens = np.min(total_tokens)

# get min ids
min_embedding_id = embedding_tokens.idxmin()
min_prompt_id = prompt_tokens.idxmin()
min_completion_id = completion_tokens.idxmin()
min_total_id = total_tokens.idxmin()

# calculate cost
prompt_costs_mini = 0.15 / 1_000_000
completion_costs_mini = 0.6 / 1_000_000

prompt_costs = 5 / 1_000_000
completion_costs = 15 / 1_000_000

avg_prompt_cost = avg_prompt_tokens * prompt_costs
avg_completion_cost = avg_completion_tokens * completion_costs
avg_total_cost = avg_prompt_cost + avg_completion_cost

max_prompt_cost = max_prompt_tokens * prompt_costs
max_completion_cost = max_completion_tokens * completion_costs
max_total_cost = max_prompt_cost + max_completion_cost

min_prompt_cost = min_prompt_tokens * prompt_costs
min_completion_cost = min_completion_tokens * completion_costs
min_total_cost = min_prompt_cost + min_completion_cost

In [26]:
token_data = [
    ["Embedding", avg_embedding_tokens, max_embedding_tokens, min_embedding_tokens, max_embedding_id, min_embedding_id, "", "", ""],
    ["Prompt", avg_prompt_tokens, max_prompt_tokens, min_prompt_tokens, max_prompt_id, min_prompt_id, avg_prompt_cost, max_prompt_cost,min_prompt_cost],
    ["Completion", avg_completion_tokens, max_completion_tokens, min_completion_tokens, max_completion_id, min_completion_id, avg_completion_cost, max_completion_cost, min_completion_cost],
    ["Total", avg_total_tokens, max_total_tokens, min_total_tokens, max_total_id, min_total_id, avg_total_cost, max_total_cost, min_total_cost],
]

print(tabulate(token_data, headers=["Type", "Average", "Maximum", "Minimum", "Max ID", "Min ID", "Avg Cost", "Max Cost", "Min Cost"], tablefmt="grid"))

+------------+-----------+-----------+-----------+----------+----------+----------------------+----------------------+-----------------------+
| Type       |   Average |   Maximum |   Minimum |   Max ID |   Min ID | Avg Cost             | Max Cost             | Min Cost              |
+============+===========+===========+===========+==========+==========+======================+======================+=======================+
| Embedding  |    17     |        28 |        10 |        6 |        2 |                      |                      |                       |
+------------+-----------+-----------+-----------+----------+----------+----------------------+----------------------+-----------------------+
| Prompt     |  1687     |      2225 |      1046 |        9 |        5 | 0.008435000000000002 | 0.011125000000000001 | 0.00523               |
+------------+-----------+-----------+-----------+----------+----------+----------------------+----------------------+-----------------------+

# How to perform t-tests

In [8]:
from scipy.stats import ttest_ind

In [9]:
df_hybrid = pd.read_csv('./logs/2024-07-29_15-32-57_sauerkraut_hero_q6_jina_custom_prompt/hybrid_sauerkraut_hero_q6_jina_custom_prompt_2024-07-29_15-32-57.csv',
                      sep=';')

In [17]:
hybrid_prompt_tokens = df_hybrid["prompt_tokens"]
print(ttest_ind(hybrid_prompt_tokens, prompt_tokens)) # equal_var=False if needed
print(f"{np.mean(hybrid_prompt_tokens)} || {avg_prompt_tokens}")

Ttest_indResult(statistic=2.899162831701088, pvalue=0.008322323384283261)
2144.75 || 1687.0


A t-test can only be performed if:
1. the data is relatively normally distributed
2. the samples have approximately equal variance

If the datasets do not have equal variance, we can use the equal_var=False flag for the t-Test.

I don't know yet what to do if the data is not normally distributed

We can test if the data follows a normal distribution with the Shapiro-Wilk test. <br>
Alternatively the D’Agostino K2 test can be used to measure Kurtosis and Skewness of a distribution.

I know I learned when to which test in Sportscience, so I might want to look that up again.

Both functions are taken from this tutorial: <br>
https://datagy.io/normality-test-python/

In [18]:
from scipy.stats import shapiro

def shapiro_test(data, alpha = 0.05):
    stat, p = shapiro(data)
    if p > alpha:
        print('Data looks Gaussian')
    else:
        print('Data look does not look Gaussian')

In [20]:
shapiro_test(prompt_tokens)

Data looks Gaussian


In [21]:
from scipy.stats import normaltest

def dagostino_test(data, alpha = 0.05):
    stat, p = normaltest(data)
    if p > alpha:
        print('Data looks Gaussian')
    else:
        print('Data does not look Gaussian')

In [22]:
shapiro_test(prompt_tokens)

Data looks Gaussian


We can check for equal variance using the Levene's test

Taken from this tutorial: <br>
https://datagy.io/python-levene-test/

In [23]:
from scipy.stats import levene

def levene_test(group1, group2):
    levene_stat, p_value = levene(group1, group2)
    if p_value < 0.05:
        print("Variances are significantly different.")
    else:
        print("Variances are likely similar.")

In [24]:
levene_test(prompt_tokens, hybrid_prompt_tokens)

Variances are likely similar.
